In [1]:
!pip install -q gradio gTTS imageio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.8 MB/s eta 0:00:00


In [2]:
%%writefile softkill_cosmos_sim_oneclick.py
import os, io, json, random, tempfile
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr

# Optional extras
try:
    from gtts import gTTS
    TTS_AVAILABLE = True
except Exception:
    TTS_AVAILABLE = False
try:
    import imageio
    GIF_AVAILABLE = True
except Exception:
    GIF_AVAILABLE = False

# ====== Cosmic Softkill-9000 Settings ======
ROLES = {
    "Longsight": "Marksman from the Vyr'khai star-clans",
    "Lifebinder": "Medic-priest of the Lumenari bioconclave",
    "Specter": "Recon shade of the Zephryl drift",
    "Whisper": "Diplomat-chorus from the Mycelian hegemony",
    "Archivist": "Sentient archive node of the Ferroth lattice",
    "Brawler": "Hand-to-hand combat specialist",
    "Armsmaster": "Weapons specialist",
    "Explosives Expert": "Specialist in demolition and ordnance",
}
ATTRS   = ["Strength", "Empathy", "Intelligence", "Mobility", "Tactical"]
ACTIONS = ["advance", "defend", "stabilise", "negotiate", "withdraw"]
SPECIES = {
    "Vyr'khai":   {"Strength": +6, "Empathy": -2, "Intelligence": +0, "Mobility": +4, "Tactical": +5},
    "Lumenari":   {"Strength": +1, "Empathy": +8, "Intelligence": +4, "Mobility": +1, "Tactical": +1},
    "Zephryl":    {"Strength": +1, "Empathy": +1, "Intelligence": +3, "Mobility": +8, "Tactical": +3},
    "Mycelian":   {"Strength": -1, "Empathy": +6, "Intelligence": +6, "Mobility": -1, "Tactical": +2},
    "Ferroth":    {"Strength": +8, "Empathy": -3, "Intelligence": +2, "Mobility": -2, "Tactical": +4},
    "Aetherborn": {"Strength": +9, "Empathy": +2, "Intelligence": +8, "Mobility": +7, "Tactical": +4},
    "Kinetari":   {"Strength": +5, "Empathy": +1, "Intelligence": +4, "Mobility": +7, "Tactical": +5},
    "Verdan":     {"Strength": +3, "Empathy": +2, "Intelligence": +5, "Mobility": +1, "Tactical": +7},
}

GALAXIES = ["Andromeda (M31)","Triangulum (M33)","Sombrero (M104)","Whirlpool (M51)",
            "Sagittarius Dwarf","Large Magellanic Cloud","Nyx Halo","Aetheric Veil",
            "Kijani Spiral","Umbral Reef","Karibu Vortex", "Centaurus A", "Messier 81",
            "Sculptor Galaxy", "Fornax Cluster", "Perseus Cluster", "Coma Cluster"]
SYL = ["ka","ru","sha","vel","dra","tor","my","cel","lum","vyr","fer","ze","phy","ae","ki","ja","ni","um","bral",
       "xon","lyr","qel","zix","vok","nar","pyl","cre","dax","jyn"]
def gen_planet(): return "".join(random.choice(SYL) for _ in range(random.randint(2,4))).title()

TERRAIN = ["Urban Lattice","Desert Glass","Ice Ridge","Oceanic Platforms","Jungle Canopy",
           "Volcanic Spires", "Acidic Swamps", "Crystal Caves", "Floating Islands"]
WEATHER = ["Clear","Ion Storm","Radiation Flare","Sandstorm","Cryo Blizzards",
           "Plasma Rain", "Gravity Eddies", "Sonic Winds", "Magnetic Anomalies"]

SCENARIOS = [
    "Refugee flotilla near a magnetar; containment fields failing.",
    "Xenofauna stampede through crystalline corridors; civilians trapped.",
    "Pirate corsairs blockading stargate; reactor leaks destabilising transit.",
    "Planetary ocean rising after moon-shear; bio-domes at risk.",
    "Clan schism; peace-talks collapsing on neutral ring-station."
]

BANTER = {
    "Longsight":[ "Vacuum steady. Pulse steadier.","Line held. Regrets pending.","One pulse—one ending—preferably not ours." ],
    "Lifebinder":[ "Biofield humming. Statistics declining.","If it bleeds, I can stabilise it.","Life prefers cooperation; let’s oblige." ],
    "Specter":[ "Ghost-walk engaged. Doors where walls used to be.","Paths mapped. Shadows compliant.","Seen and unseen are tactical categories." ],
    "Whisper":[ "Words first, wounds last.","Lower the heat; raise the harmony.","Consent acquired. Conflict retired." ],
    "Archivist":[ "Recording. Remembering. History has teeth.","Truth cached. Lies quarantined.","Ethics subroutines purring. Do better." ],
    "Brawler":[ "Close quarters. Good.","My hands are registered weapons.","Less talk, more impact." ],
    "Armsmaster":[ "Ordnance prepped.","Picking the right tool for the job.","Let the weapon do the talking." ],
    "Explosives Expert":[ "Charge set. Stand clear.","Demolitions are a delicate art.","Making exits where there weren't any." ],
}

BASE_REWARD = {
    "Longsight":{"advance":8,"defend":6,"stabilise":-3,"negotiate":-4,"withdraw":-1},
    "Lifebinder":{"advance":-1,"defend":3,"stabilise":10,"negotiate":1,"withdraw":2},
    "Specter":{"advance":7,"defend":3,"stabilise":-1,"negotiate":0,"withdraw":4},
    "Whisper":{"advance":-2,"defend":1,"stabilise":1,"negotiate":10,"withdraw":3},
    "Archivist":{"advance":0,"defend":2,"stabilise":2,"negotiate":6,"withdraw":3},
    "Brawler":{"advance":7,"defend":5,"stabilise":-2,"negotiate":-3,"withdraw":2},
    "Armsmaster":{"advance":6,"defend":7,"stabilise":-1,"negotiate":-2,"withdraw":3},
    "Explosives Expert":{"advance":5,"defend":4,"stabilise":-4,"negotiate":-3,"withdraw":1},
}
SCENARIO_MOD = {
    "magnetar":{"withdraw":+2,"defend":+1},
    "xenofauna":{"stabilise":+3,"defend":+2},
    "pirate":{"advance":+2,"defend":+1,"negotiate":+1},
    "ocean":{"stabilise":+2,"withdraw":+1},
    "schism":{"negotiate":+4,"defend":+1},
}
TERRAIN_MOD = {
    "Urban Lattice":{"defend":+1,"negotiate":+1},
    "Desert Glass":{"withdraw":+1,"advance":-1},
    "Ice Ridge":{"defend":+1,"stabilise":+1},
    "Oceanic Platforms":{"stabilise":+2,"withdraw":+1},
    "Jungle Canopy":{"advance":+1,"defend":0},
    "Volcanic Spires": {"Strength":+2, "defend":+1},
    "Acidic Swamps": {"Mobility":+1, "withdraw":+2},
    "Crystal Caves": {"Intelligence":+1, "Tactical":+2},
    "Floating Islands": {"Mobility":+2, "stabilise":+1},
}
WEATHER_MOD = {
    "Clear":{},
    "Ion Storm":{"withdraw":+2,"defend":+1},
    "Radiation Flare":{"withdraw":+2,"stabilise":+1},
    "Sandstorm":{"withdraw":+1,"defend":+1},
    "Cryo Blizzards":{"defend":+1,"stabilise":+1},
    "Plasma Rain": {"Strength":+1, "defend":+2},
    "Gravity Eddies": {"Mobility":+2, "stabilise":+1},
    "Sonic Winds": {"Intelligence":+1, "Tactical":+2},
    "Magnetic Anomalies": {"Strength":+1, "withdraw":+2},
}
ETHICS = {"save_civilian":+8,"collateral":-8,"document":+3,"deescalate":+5}

def ethics_bonus(role, action, key):
    e = 0
    if key=="magnetar" and action in ["withdraw","defend"]: e += ETHICS["save_civilian"]
    if key=="xenofauna" and action=="stabilise": e += ETHICS["save_civilian"]
    if key=="pirate" and action in ["negotiate","defend"]: e += ETHICS["deescalate"]
    if key=="ocean" and action in ["stabilise","advance"]: e += ETHICS["save_civilian"]
    if key=="schism" and action=="negotiate": e += ETHICS["deescalate"]
    if role=="Archivist" and action in ["defend","negotiate"]: e += ETHICS["document"]
    if role == "Brawler" and action == "defend": e += ETHICS["save_civilian"]
    if role == "Armsmaster" and action == "advance" in key == "pirate": e += ETHICS["deescalate"]
    if role == "Explosives Expert" and action == "withdraw" in key == "magnetar": e += ETHICS["save_civilian"]
    return e

def infer_key(text):
    low=text.lower()
    for k in SCENARIO_MOD:
        if k in low: return k
    return ""

def env_reward(role, action, scenario, terrain, weather, morals=True):
    base = BASE_REWARD[role][action]
    mod = 0
    key = infer_key(scenario)
    mod += SCENARIO_MOD.get(key,{}).get(action,0)
    mod += TERRAIN_MOD.get(terrain,{}).get(action,0)
    mod += WEATHER_MOD.get(weather,{}).get(action,0)
    e = ethics_bonus(role, action, key) if morals else 0
    noise = np.random.uniform(-1.5,1.5)
    return base + mod + e + noise

# Q-learning for marksman
def train_longsight_q(episodes=1000, gamma=0.90, alpha=0.3, eps=0.2, morals=True):
    keys = list(SCENARIO_MOD.keys())
    n_states, n_actions = len(keys), len(ACTIONS)
    Q = np.zeros((n_states, n_actions))
    for _ in range(episodes):
        s = np.random.randint(0, n_states)
        a = np.random.randint(0, n_actions) if np.random.rand()<eps else int(np.argmax(Q[s]))
        r = env_reward("Longsight", ACTIONS[a], keys[s], random.choice(TERRAIN), random.choice(WEATHER), morals)
        Q[s,a] = (1-alpha)*Q[s,a] + alpha*(r + gamma*np.max(Q[s]))
    return Q

def choose_action(role, scenario, terrain, weather, Q=None, morals=True):
    if role=="Longsight" and Q is not None:
        keys=list(SCENARIO_MOD.keys()); idx=0
        low=scenario.lower()
        for i,k in enumerate(keys):
            if k in low: idx=i; break
        return ACTIONS[int(np.argmax(Q[idx]))]
    # Default action choices for other roles
    if role=="Lifebinder":
        return "stabilise" if any(k in scenario.lower() for k in ["ocean","xenofauna","magnetar"]) else "defend"
    if role=="Whisper":
        return "negotiate" if any(k in scenario.lower() for k in ["schism","pirate"]) else "defend"
    if role=="Specter":
        return "advance" if "pirate" in scenario.lower() else "defend"
    if role=="Archivist":
        return "negotiate" if any(k in scenario.lower() for k in ["schism","pirate"]) else "defend"
    if role=="Brawler":
        return "advance" if any(k in scenario.lower() for k in ["xenofauna", "pirate"]) else "defend"
    if role == "Armsmaster":
        return "defend" if any(k in scenario.lower() for k in ["pirate", "magnetar"]) else "advance"
    if role == "Explosives Expert":
        return "advance" if any(k in scenario.lower() for k in ["pirate"]) else "defend"
    return random.choice(ACTIONS)


def radar_fig(stats):
    labels=list(next(iter(stats.values())).keys())
    n=len(labels); angles=np.linspace(0,2*np.pi,n,endpoint=False).tolist(); angles+=angles[:1]
    fig,ax=plt.subplots(figsize=(8,8),subplot_kw=dict(polar=True))
    for role,vals in stats.items():
        data=list(vals.values()); data+=data[:1]
        ax.plot(angles,data,label=role); ax.fill(angles,data,alpha=0.15)
    ax.set_thetagrids(np.degrees(angles[:-1]),labels); ax.set_ylim(0,150) # Increased y-limit
    ax.set_title("Squad Capability Radar", y=1.1); ax.legend(loc="upper right",bbox_to_anchor=(1.25,1.1))
    fig.tight_layout(); return fig

def make_gif_path(trails):
    if not GIF_AVAILABLE: return None
    import imageio
    frames=[]
    for t in range(len(next(iter(trails.values())))):
        fig,ax=plt.subplots(figsize=(4,4))
        ax.set_xlim(0,1); ax.set_ylim(0,1); ax.set_xticks([]); ax.set_yticks([])
        for role,trail in trails.items():
            x,y=trail[t]
            ax.plot([p[0] for p in trail[:t+1]],[p[1] for p in trail[:t+1]])
            ax.scatter([x],[y]); ax.text(x,y,role,fontsize=9)
        ax.set_title("Mission Map — Timeline"); fig.tight_layout()
        buf=io.BytesIO(); fig.savefig(buf,format="png",dpi=120); plt.close(fig); buf.seek(0)
        frames.append(imageio.v2.imread(buf))
    tmp=tempfile.NamedTemporaryFile(delete=False,suffix=".gif")
    imageio.mimsave(tmp.name, frames, duration=0.15)
    return tmp.name

def gen_tts_mp3(lines):
    if not TTS_AVAILABLE: return None
    txt=" | ".join(lines[-5:]) if lines else "Mission starting."
    mp=tempfile.NamedTemporaryFile(delete=False,suffix=".mp3").name
    try:
        gTTS(text=txt,lang="en").save(mp); return mp
    except Exception:
        return None

def run_mission(galaxy_input, planet_name_input, terrain_input, weather_input, morals,
                  long_sp, life_sp, spec_sp, whis_sp, arch_sp, brawler_sp, armsmaster_sp, explosives_expert_sp,
                  steps, ls_train_eps,
                  b_str, b_emp, b_int, b_mob, b_tac):

      # Randomize environmental parameters
      galaxy = random.choice(GALAXIES)
      planet = f"{gen_planet()}-{random.randint(1,999)}"
      terrain = random.choice(TERRAIN)
      weather = random.choice(WEATHER)


      base=dict(zip(ATTRS,[b_str,b_emp,b_int,b_mob,b_tac]))
      def apply_species(spec):
          mods=SPECIES[spec]; return {a:max(0,min(110,base[a]+mods[a])) for a in ATTRS}
      stats={
          "Longsight": apply_species(long_sp),
          "Lifebinder": apply_species(life_sp),
          "Specter": apply_species(spec_sp),
          "Whisper": apply_species(whis_sp),
          "Archivist": apply_species(arch_sp),
          "Brawler": apply_species(brawler_sp),
          "Armsmaster": apply_species(armsmaster_sp),
          "Explosives Expert": apply_species(explosives_expert_sp),
      }


      Q=train_longsight_q(episodes=int(ls_train_eps), morals=morals)

      steps=int(steps)
      log=[f"MISSION: {galaxy} // Planet {planet} // Terrain={terrain} // Weather={weather} // Morals={'ON' if morals else 'OFF'} // Steps={steps}"]


      rewards={r:[] for r in stats}; cum={r:0 for r in stats}
      trails={r:[] for r in stats}; pos={r:(random.random()*0.2+0.4, random.random()*0.2+0.4) for r in stats}

      for t in range(steps):
          scenario=random.choice(SCENARIOS)
          for role in stats:
              action=choose_action(role, scenario, terrain, weather, Q, morals)
              r=env_reward(role, action, scenario, terrain, weather, morals)
              r*=(0.5 + stats[role]["Strength"]/200.0)
              cum[role]+=r; rewards[role].append(cum[role])
              x,y=pos[role]; step_mag=(stats[role]["Mobility"]/200.0)*0.1 # Use stats dictionary
              x=min(1,max(0,x+np.random.uniform(-step_mag,step_mag)))
              y=min(1,max(0,y+np.random.uniform(-step_mag,step_mag)))
              pos[role]=(x,y); trails[role].append((x,y))
              quip=random.choice(BANTER[role])
              log.append(f"[{t:02d}] {role}: {quip} | Action={action} | Scenario={scenario} | Δ={r:.1f}")

      radar=radar_fig(stats)
      fig2,ax2=plt.subplots(figsize=(8,5))
      for role in stats: ax2.plot(rewards[role],label=role)
      ax2.set_title("Cumulative Reward // Cosmic Mission"); ax2.set_xlabel("Ticks"); ax2.set_ylabel("Cumulative Reward")
      ax2.grid(True); ax2.legend(); curve=fig2

      gif_path=make_gif_path(trails)
      audio_path=gen_tts_mp3(log)

      export={"galaxy":galaxy,"planet":planet,"terrain":terrain,"weather":weather,"morals":morals,
              "species":{"Longsight":long_sp,"Lifebinder":life_sp,"Specter":spec_sp,"Whisper":whis_sp,"Archivist":arch_sp,"Brawler":brawler_sp,"Armsmaster":armsmaster_sp,"Explosives Expert":explosives_expert_sp},
              "stats":stats}
      export_json=json.dumps(export,indent=2)

      return radar, curve, (gif_path if gif_path else None), "\n".join(log[-250:]), export_json, (audio_path if audio_path else None)

def build_app():
      with gr.Blocks() as demo:
          gr.Markdown("# 🌌 SOFTKILL-9000 — Cosmic Mission Simulator (One-Click Demo)")
          gr.Markdown("Alien squad • Ethics-aware RL • Terrain & Weather • Timeline animation • Voice banter")

          with gr.Row():
              # These inputs will be ignored for randomization but kept for interface structure
              galaxy=gr.Dropdown(GALAXIES, value="Nyx Halo", label="Galaxy")
              planet=gr.Textbox(value="", label="Custom Planet (optional)")
          with gr.Row():
               # These inputs will be ignored for randomization but kept for interface structure
              terrain=gr.Dropdown(["Urban Lattice","Desert Glass","Ice Ridge","Oceanic Platforms","Jungle Canopy",
                                   "Volcanic Spires", "Acidic Swamps", "Crystal Caves", "Floating Islands"],
                                  value="Urban Lattice", label="Terrain")
              # These inputs will be ignored for randomization but kept for interface structure
              weather=gr.Dropdown(["Clear","Ion Storm","Radiation Flare","Sandstorm","Cryo Blizzards",
                                   "Plasma Rain", "Gravity Eddies", "Sonic Winds", "Magnetic Anomalies"],
                                  value="Clear", label="Weather")
              morals=gr.Checkbox(value=True, label="Enable Ethics Shaping")

          gr.Markdown("### Squad Species")
          with gr.Row():
             long_sp=gr.Dropdown(list(SPECIES.keys()), value="Vyr'khai",label="Longsight")
             life_sp=gr.Dropdown(list(SPECIES.keys()), value="Lumenari",label="Lifebinder")
             spec_sp=gr.Dropdown(list(SPECIES.keys()), value="Zephryl",label="Specter")
             whis_sp=gr.Dropdown(list(SPECIES.keys()), value="Mycelian",label="Whisper")
             arch_sp=gr.Dropdown(list(SPECIES.keys()), value="Ferroth",label="Archivist")
             brawler_sp=gr.Dropdown(list(SPECIES.keys()), value="Aetherborn",label="Brawler")
             armsmaster_sp=gr.Dropdown(list(SPECIES.keys()), value="Kinetari",label="Armsmaster")
             explosives_expert_sp=gr.Dropdown(list(SPECIES.keys()), value="Verdan",label="Explosives Expert")



          gr.Markdown("### Base Attributes (before species modifiers)")
          with gr.Row():
              b_str=gr.Slider(0,100,60,label="Base Strength")
              b_emp=gr.Slider(0,100,60,label="Base Empathy")
              b_int=gr.Slider(0,100,60,label="Base Intelligence")
              b_mob=gr.Slider(0,100,60,label="Base Mobility")
              b_tac=gr.Slider(0,100,60,label="Base Tactical")

          with gr.Row():
              steps=gr.Slider(10,150,value=60,step=5,label="Mission Ticks")
              eps  =gr.Slider(0,3000,value=1000,step=50,label="Longsight Training Episodes (Q-learning)")

          run_btn=gr.Button("🚀 Execute Mission")
          out_radar=gr.Plot(label="Capability Radar")
          out_curve=gr.Plot(label="Cumulative Reward Curve")
          out_gif  =gr.Video(label="Mission Timeline Animation (GIF)")
          out_log  =gr.Textbox(lines=18,label="Banter & Mission Log")
          out_json =gr.Textbox(lines=12,label="Export JSON")
          out_audio=gr.Audio(label="Squad Voice Lines (TTS)", interactive=False)

          def _run(gal,pl,terr,weath,mo,sp1,sp2,sp3,sp4,sp5,sp6,sp7,s8,stps,ep,bs,be,bi,bm,bt):

              return run_mission(gal,pl,terr,weath,mo,sp1,sp2,sp3,sp4,sp5,sp6,sp7,s8,stps,ep,bs,be,bi,bm,bt)

          run_btn.click(_run,
              [galaxy,planet,terrain,weather,morals, long_sp, life_sp, spec_sp, whis_sp, arch_sp, brawler_sp, armsmaster_sp, explosives_expert_sp, steps, eps, b_str, b_emp, b_int, b_mob, b_tac],
              [out_radar,out_curve,out_gif,out_log,out_json,out_audio])
      return demo

demo = build_app()

if __name__ == "__main__":
    demo.launch(share=False)

Writing softkill_cosmos_sim_oneclick.py


In [3]:
import softkill_cosmos_sim_oneclick as app
# app.demo.close() # Close the previous Gradio app
app.demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a9ec36f764551b8ee9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
